we have to user the same pre-processing step like the transformer pre-trained .
So import the Tokenizer according to the model you choose and use it.


In [6]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [7]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")


print(inputs)

{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
        12172,  2607,  2026,  2878,  2166,  1012,   102],
       [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,
            0,     0,     0,     0,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>}


In [8]:
tokenizer.batch_decode(inputs["input_ids"])

["[CLS] i've been waiting for a huggingface course my whole life. [SEP]",
 '[CLS] i hate this so much! [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]']

# going through the model

In [8]:
from transformers import TFAutoModel # specific to tensorflow
 
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModel.from_pretrained(checkpoint)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


the output of a model will be high dimentionality vector:

A high-dimensional vector?
The vector output by the Transformer module is usually large. It generally has three dimensions:

- Batch size: The number of sequences processed at a time (2 in our example).
- Sequence length: The length of the numerical representation of the sequence (16 in our example).
- Hidden size: The vector dimension of each model input.



In [9]:
outputs = model(inputs)
print(outputs.last_hidden_state.shape) # can also be accessed as outputs["last_hidden_state"]

(2, 16, 768)


### models gives logit only not probabilites

In [10]:
outputs

TFBaseModelOutput(last_hidden_state=<tf.Tensor: shape=(2, 16, 768), dtype=float32, numpy=
array([[[-0.17978016,  0.23332867,  0.6320988 , ..., -0.30166638,
          0.50082004,  0.14814392],
        [ 0.27577713,  0.64971256,  0.31997722, ..., -0.07599576,
          0.5136177 ,  0.13292262],
        [ 0.90458494,  0.09851397,  0.29497266, ...,  0.33519477,
         -0.14074144, -0.6464032 ],
        ...,
        [ 0.14658965,  0.56606036,  0.32352817, ..., -0.33757502,
          0.5099784 , -0.05610833],
        [ 0.7500042 ,  0.04872625,  0.17379995, ...,  0.46841478,
          0.00296644, -0.6083757 ],
        [ 0.05194428,  0.37294865,  0.5223328 , ...,  0.35840556,
          0.6500425 , -0.38829762]],

       [[-0.29370666,  0.7282564 , -0.14972728, ..., -0.11868107,
         -1.0226725 , -0.04215715],
        [-0.22063628,  0.93838453, -0.09512515, ..., -0.36431694,
         -0.66052234,  0.24069722],
        [-0.15360793,  0.8987503 , -0.0727645 , ..., -0.21891795,
         -0.8

<img src="/home/pavithra/projects/Natural-Language-Processing/learnings/HuggingFace/2_using_transformars/0_model_architecture.png" width="1500" height="1000">


the following models are differ in the "head" position

This particular model will be following a architecture, there are many like that for a specific task:
- *Model (retrieve the hidden states)
- *ForCausalLM
- *ForMaskedLM
- *ForMultipleChoice
- *ForQuestionAnswering
- *ForSequenceClassification
- *ForTokenClassification
- and others 🤗

In [9]:
# lets use a classcification model
from transformers import TFAutoModelForSequenceClassification
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(inputs)
print(outputs.logits.shape)


2024-05-26 00:36:26.245616: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


(2, 2)


In [14]:

print(outputs.logits)
# you can convert it to softmax.
import tensorflow as tf
prediction = tf.math.softmax(outputs.logits, axis=-1)
print(prediction)


tf.Tensor(
[[-1.5606965  1.6122816]
 [ 4.1692314 -3.346448 ]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[4.0195372e-02 9.5980465e-01]
 [9.9945587e-01 5.4418371e-04]], shape=(2, 2), dtype=float32)


In [15]:
model.config.id2label


{0: 'NEGATIVE', 1: 'POSITIVE'}